In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from random import randint
import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch
from sklearn import *

Datasets:
https://scikit-learn.org/stable/datasets/toy_dataset.html
### Diabetes Dataset

In [4]:
X, Y = datasets.load_diabetes(return_X_y=True)
print(X.shape, Y.shape)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,Y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(442, 10) (442,)
(353, 10) (89, 10) (353,) (89,)


In [13]:
def my_func(config, reporter):  # add the reporter parameter
      # report the configuration
    clf = sklearn.neural_network.MLPRegressor(solver='lbfgs', alpha=config["alpha"],
                                            hidden_layer_sizes=(5,2),
                                            max_iter=90000,activation='relu',random_state=1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    reporter(config, mean_loss = sklearn.metrics.mean_squared_error(y_test, y_pred))

In [ ]:
tune.register_trainable("my_func", my_func)
ray.shutdown()
ray.init()

res = tune.run_experiments({
    "my_experiment": {
        "run": "my_func",
        "stop": { "mean_accuracy": 4000 },
        "config": {
            "alpha": tune.grid_search([0.1, 0.2]),
            "hidden_layer_sizes": tune.grid_search([(5,2), (10,2)]),
        }
    }
})

In [14]:
algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
algo = ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = AsyncHyperBandScheduler()
analysis = tune.run(
        my_func,
        name="my_exp",
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=10,
        config={
            "steps": 100,
            "alpha": tune.uniform(0.0001, 10),
        },
    )

print("Best hyperparameters found were: ", analysis.best_config)

2022-04-07 01:19:39,639	WARNING bayesopt.py:397 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.


Trial name,status,loc,alpha
my_func_0841d2b6,RUNNING,,3.74546


Result for my_func_08593dba:
  _metric:
    alpha: 9.507147992668521
    steps: 100
  date: 2022-04-07_01-19-44
  done: false
  experiment_id: b7de1dfffefc43619770959d150e5e69
  hostname: Ghallab
  iterations_since_restore: 1
  mean_loss: 3022.6462792488355
  neg_mean_loss: -3022.6462792488355
  node_ip: 192.168.1.106
  pid: 1920
  time_since_restore: 0.08479690551757812
  time_this_iter_s: 0.08479690551757812
  time_total_s: 0.08479690551757812
  timestamp: 1649287184
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 08593dba
  
Result for my_func_08593dba:
  _metric:
    alpha: 9.507147992668521
    steps: 100
  date: 2022-04-07_01-19-44
  done: true
  experiment_id: b7de1dfffefc43619770959d150e5e69
  experiment_tag: 2_alpha=9.5071,steps=100
  hostname: Ghallab
  iterations_since_restore: 1
  mean_loss: 3022.6462792488355
  neg_mean_loss: -3022.6462792488355
  node_ip: 192.168.1.106
  pid: 1920
  time_since_restore: 0.08479690551757812
  time_this_iter_s: 0.08479690551

(pid=1920) Windows fatal exception: access violation
(pid=1920) 
(pid=1696) Windows fatal exception: access violation
(pid=1696) 


Trial name,status,loc,alpha,loss,iter,total time (s),neg_mean_loss
my_func_086d7b32,RUNNING,,7.31997,,,,
my_func_0af7fb2c,RUNNING,,5.98662,,,,
my_func_0afd4ebb,PENDING,,1.56027,,,,
my_func_0841d2b6,TERMINATED,,3.74546,3018.19,1,0.213265,-3018.19
my_func_08593dba,TERMINATED,,9.50715,3022.65,1,0.0847969,-3022.65


...

Result for my_func_119975bc:
  _metric:
    alpha: 7.080754970702675
    steps: 100
  date: 2022-04-07_01-20-08
  done: true
  experiment_id: 3173e16acff04f16aefea05e974922bc
  hostname: Ghallab
  iterations_since_restore: 1
  mean_loss: 3020.46321163851
  neg_mean_loss: -3020.46321163851
  node_ip: 192.168.1.106
  pid: 4244
  time_since_restore: 0.12169384956359863
  time_this_iter_s: 0.12169384956359863
  time_total_s: 0.12169384956359863
  timestamp: 1649287208
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 119975bc
  


(pid=4244) Windows fatal exception: access violation
(pid=4244) 


Trial name,status,loc,alpha,loss,iter,total time (s),neg_mean_loss
my_func_0841d2b6,TERMINATED,,3.74546,3018.19,1,0.213265,-3018.19
my_func_08593dba,TERMINATED,,9.50715,3022.65,1,0.0847969,-3022.65
my_func_086d7b32,TERMINATED,,7.31997,3020.87,1,0.748515,-3020.87
my_func_0af7fb2c,TERMINATED,,5.98662,3020.1,1,0.50317,-3020.1
my_func_0afd4ebb,TERMINATED,,1.56027,3016.68,1,0.67725,-3016.68
my_func_0e0738b0,TERMINATED,,1.56003,3016.78,1,0.471735,-3016.78
my_func_10c6ae0d,TERMINATED,,0.58093,3016.02,1,0.103507,-3016.02
my_func_10fcef92,TERMINATED,,8.66177,3021.61,1,0.463764,-3021.61
my_func_11591665,TERMINATED,,6.01119,3019.8,1,0.546057,-3019.8
my_func_119975bc,TERMINATED,,7.08075,3020.46,1,0.121694,-3020.46


2022-04-07 01:20:08,944	INFO tune.py:561 -- Total run time: 29.31 seconds (29.13 seconds for the tuning loop).


Best hyperparameters found were:  {'steps': 100, 'alpha': 0.5809303133207778}
